In [1]:
from pathlib import Path
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import seaborn as sns
sns.set_theme()
import math
import pandas as pd
import random
import numpy as np
import sys
import shutil

import IMP
import IMP.atom

sys.path.append(str(Path(Path.home(), "Documents/xray/src")))
from refine import refine_hs_to_max_ff

In [7]:
input_file = Path(Path.home(), "Documents/xray/dev/35_cif_combos/data/7mhf.csv")
input_df = pd.read_csv(input_file, index_col=0)
input_df.head()

,cifs,refs,ref_occs
0,/wynton/home/sali/mhancock/xray/data/cifs/7mhf...,/wynton/home/sali/mhancock/xray/data/pdbs/7mhf...,1
1,/wynton/home/sali/mhancock/xray/data/cifs/7mhf...,/wynton/home/sali/mhancock/xray/data/pdbs/7mhf...,1
2,/wynton/home/sali/mhancock/xray/data/cifs/7mhf...,/wynton/home/sali/mhancock/xray/data/pdbs/7mhf...,1
3,/wynton/home/sali/mhancock/xray/data/cifs/7mhf...,/wynton/home/sali/mhancock/xray/data/pdbs/7mhf...,1
4,/wynton/home/sali/mhancock/xray/data/cifs/7mhf...,/wynton/home/sali/mhancock/xray/data/pdbs/7mhf...,1


In [9]:
for i in range(len(input_df)):
    cif_entry = input_df.loc[i, "cifs"]
    cifs = cif_entry.split(",")

    ref_entry = input_df.loc[i, "refs"]
    refs = ref_entry.split(",")

    for j in range(len(cifs)):
        input_df.loc[i, "cif_{}".format(j)] = cifs[j]
        input_df.loc[i, "ref_{}".format(j)] = refs[j]

        input_df.loc[i, "w_0_{}".format(j)] = 1


In [10]:
input_df.drop(columns=["cifs", "refs", "ref_occs"], inplace=True)
input_df.head()


,cif_0,ref_0,w_0_0,cif_1,ref_1,w_0_1,cif_2,ref_2,w_0_2,cif_3,ref_3,w_0_3,cif_4,ref_4,w_0_4,cif_5,ref_5,w_0_5
0,/wynton/home/sali/mhancock/xray/data/cifs/7mhf...,/wynton/home/sali/mhancock/xray/data/pdbs/7mhf...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/wynton/home/sali/mhancock/xray/data/cifs/7mhf...,/wynton/home/sali/mhancock/xray/data/pdbs/7mhf...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/wynton/home/sali/mhancock/xray/data/cifs/7mhf...,/wynton/home/sali/mhancock/xray/data/pdbs/7mhf...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/wynton/home/sali/mhancock/xray/data/cifs/7mhf...,/wynton/home/sali/mhancock/xray/data/pdbs/7mhf...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/wynton/home/sali/mhancock/xray/data/cifs/7mhf...,/wynton/home/sali/mhancock/xray/data/pdbs/7mhf...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
input_df.to_csv(str(input_file)+".test.csv")